As respostas da **Questão 1** ("Qual a média de valor total (total\_amount) recebido em um mês
considerando todos os yellow táxis da frota?") estão na **célula 6** e as respostas da **questão 2** ("Qual a média de passageiros (passenger\_count) por cada hora do dia
que pegaram táxi no mês de maio considerando todos os táxis da
frota?") estão na **célula 32 (ultima célula)** . 

deixo todas as análises feitas para avaliarem a linha de raciocínio.

Obrigado por avaliarem!

In [0]:
%sql
select * from curated_nyc_taxi.yellow_taxi_curated
limit 5

VendorID,passenger_count,total_amount,tpep_pickup_datetime,tpep_dropoff_datetime,taxi_color
2,1.0,11.1,2023-03-01T00:06:43.000Z,2023-03-01T00:16:43.000Z,yellow_taxi
2,2.0,76.49,2023-03-01T00:08:25.000Z,2023-03-01T00:39:30.000Z,yellow_taxi
1,0.0,28.05,2023-03-01T00:15:04.000Z,2023-03-01T00:29:26.000Z,yellow_taxi
1,1.0,24.7,2023-03-01T00:49:37.000Z,2023-03-01T01:01:05.000Z,yellow_taxi
2,1.0,14.64,2023-03-01T00:08:04.000Z,2023-03-01T00:11:06.000Z,yellow_taxi


**Verificando se há nulos no total_amount do yellow taxi**

In [0]:
%sql
select
count(1)
from curated_nyc_taxi.yellow_taxi_curated
where total_amount is null

count(1)
0


Resolução Q1

In [0]:
%sql
-- questão 1
select
date_trunc('MONTH',tpep_pickup_datetime) as month,
ROUND(AVG(total_amount), 2) AS media_valor_total
from curated_nyc_taxi.yellow_taxi_curated
group by 1
order by 1

month,media_valor_total
2023-01-01T00:00:00.000Z,27.02
2023-02-01T00:00:00.000Z,26.9
2023-03-01T00:00:00.000Z,27.8
2023-04-01T00:00:00.000Z,28.27
2023-05-01T00:00:00.000Z,28.96


**> Resposta q1: A média mensal pode variar entre 26.9 e 28.96 dolares**

**Resolução Q2:**

verificando valores nulos na tabela yellow taxi

In [0]:
%sql
select 
count(1)
from curated_nyc_taxi.yellow_taxi_curated
where tpep_pickup_datetime >= date '2023-05-01'
and tpep_pickup_datetime <  date '2023-06-01'
and passenger_count is null

count(1)
101796


**verificando valores não nulos na tabela yellow taxi**

In [0]:
%sql
select 
count(1)
from curated_nyc_taxi.yellow_taxi_curated
where tpep_pickup_datetime >= date '2023-05-01'
and tpep_pickup_datetime <  date '2023-06-01'
and passenger_count is not null

count(1)
3411849


**CONSIDERAÇÕES**

mesmo fazendo o coalesce na camada de preprocessamento com os valores da coluna _rescued_data , ainda temos 101.796 registros nulos no campo passenger_count contra 3.411.849 não nulos no df de yellow_taxi. o que representa 2,98% de valores nulos nessa coluna. Sabemos que dados de corridas de taxi são passíveis de sazonalidade, e para não perdê-la na análise, vale a pena imputar nos nulos a média do período



In [0]:
%sql
--descobrindo a média de passenger_count dos yellow_taxis sem valor nulo, APENAS NO PERÍODO SOLICITADO
select 
round(avg(passenger_count),2) as media_passageiros_yellow
from curated_nyc_taxi.yellow_taxi_curated
where tpep_pickup_datetime >= date '2023-05-01'
and tpep_pickup_datetime <  date '2023-06-01'
and passenger_count is not null

media_passageiros_yellow
1.36


In [0]:
%sql
select *,
coalesce(passenger_count, 1.36) as passenger_count_imputed 
from curated_nyc_taxi.yellow_taxi_curated
where tpep_pickup_datetime >= timestamp '2023-05-01'
and tpep_pickup_datetime < timestamp '2023-06-01'
limit 5

VendorID,passenger_count,total_amount,tpep_pickup_datetime,tpep_dropoff_datetime,taxi_color,passenger_count_imputed
1,0.0,51.65,2023-05-01T00:33:13.000Z,2023-05-01T00:53:01.000Z,yellow_taxi,0.0
1,2.0,57.15,2023-05-01T00:42:49.000Z,2023-05-01T01:11:18.000Z,yellow_taxi,2.0
1,2.0,64.2,2023-05-01T00:56:34.000Z,2023-05-01T01:13:39.000Z,yellow_taxi,2.0
2,1.0,47.09,2023-05-01T00:00:52.000Z,2023-05-01T00:20:12.000Z,yellow_taxi,1.0
1,0.0,59.15,2023-05-01T00:05:50.000Z,2023-05-01T00:19:41.000Z,yellow_taxi,0.0


**criando tabela temporária yellow taxi com o coalesce da média nos passenger_count null**

In [0]:
df_y_med_pas_count = spark.sql(
  "select *, "
  "coalesce(passenger_count, 1.36) as passenger_count_imputed "
  "from curated_nyc_taxi.yellow_taxi_curated "
  "where tpep_pickup_datetime >= timestamp '2023-05-01' "
  "and tpep_pickup_datetime < timestamp '2023-06-01'"
)
df_y_med_pas_count.createOrReplaceTempView("df_y_med_pas_count")

**Agora fazendo o mesmo processo para o green taxi**

In [0]:
%sql
select 
count(1)
from curated_nyc_taxi.green_taxi_curated
where lpep_pickup_datetime >= date '2023-05-01'
and lpep_pickup_datetime <  date '2023-06-01'
and passenger_count is null

count(1)
4732


In [0]:
%sql
select 
count(1)
from curated_nyc_taxi.green_taxi_curated
where lpep_pickup_datetime >= date '2023-05-01'
and lpep_pickup_datetime <  date '2023-06-01'
and passenger_count is not null

count(1)
64436


**CONSIDERAÇÕES**

mesmo fazendo o coalesce na camada de preprocessamento com os valores da coluna _rescued_data , ainda temos 4732 registros nulos no campo passenger_count contra 64436 não nulos no df de green_taxi. o que representa 7,34% de valores nulos nessa coluna. Sabemos que dados de corridas de taxi são passíveis de sazonalidade, e para não perdê-la na análise, vale a pena imputar nos nulos a média do período

In [0]:
%sql
--descobrindo a média de passenger_count dos green_taxis sem valor nulo, APENAS NO PERÍODO SOLICITADO
select 
round(avg(passenger_count),2) as media_passageiros_green
from curated_nyc_taxi.green_taxi_curated
where lpep_pickup_datetime >= date '2023-05-01'
and lpep_pickup_datetime <  date '2023-06-01'
and passenger_count is not null

media_passageiros_green
1.27


In [0]:
%sql
select *,
coalesce(passenger_count, 1.27) as passenger_count_imputed 
from curated_nyc_taxi.green_taxi_curated
where lpep_pickup_datetime >= timestamp '2023-05-01'
and lpep_pickup_datetime < timestamp '2023-06-01'
limit 5

VendorID,passenger_count,total_amount,lpep_pickup_datetime,lpep_dropoff_datetime,taxi_color,passenger_count_imputed
2,1.0,31.4,2023-05-01T00:52:10.000Z,2023-05-01T01:05:26.000Z,green_taxi,1.0
2,1.0,40.55,2023-05-01T00:29:49.000Z,2023-05-01T00:50:11.000Z,green_taxi,1.0
2,1.0,14.16,2023-05-01T00:25:19.000Z,2023-05-01T00:32:12.000Z,green_taxi,1.0
2,1.0,32.57,2023-05-01T00:07:06.000Z,2023-05-01T00:27:33.000Z,green_taxi,1.0
2,1.0,9.0,2023-05-01T00:43:31.000Z,2023-05-01T00:46:59.000Z,green_taxi,1.0


criando tabela temporária green taxi com o coalesce da média nos passenger_count null

In [0]:
df_g_med_pas_count = spark.sql(
  "select *, "
  "coalesce(passenger_count, 1.27) as passenger_count_imputed "
  "from curated_nyc_taxi.green_taxi_curated "
  "where lpep_pickup_datetime >= timestamp '2023-05-01' "
  "and lpep_pickup_datetime < timestamp '2023-06-01'"
)
df_g_med_pas_count.createOrReplaceTempView("df_g_med_pas_count")

In [0]:
df_y_med_pas_count

DataFrame[VendorID: bigint, passenger_count: double, total_amount: double, tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp, taxi_color: string, passenger_count_imputed: double]

**Analise individual da quantidade de passageiros por hora no yellow taxi no mes de Maio**

In [0]:
%sql
-- passenger count yellow taxi 
select
hour(tpep_pickup_datetime) AS hora,
SUM(COALESCE(passenger_count)) as qtd_passageiros
from df_y_med_pas_count
where  tpep_pickup_datetime >= timestamp '2023-05-01'
and tpep_pickup_datetime <  timestamp '2023-06-01'
group by 1
order by 1

hora,qtd_passageiros
0,128112.0
1,83857.0
2,54693.0
3,35650.0
4,22646.0
5,23719.0
6,57865.0
7,118318.0
8,163493.0
9,186022.0


**## Analise individual da quantidade de passageiros por hora no green taxi**

In [0]:
%sql
-- passenger count green taxi
select
hour(lpep_pickup_datetime) AS hora,
round(sum(coalesce(passenger_count_imputed, 0)),0) as qtd_passageiros
from df_g_med_pas_count
where  lpep_pickup_datetime >= date '2023-05-01'
and lpep_pickup_datetime <  date '2023-06-01'
group by 1
order by 1

hora,qtd_passageiros
0,1612.0
1,1061.0
2,830.0
3,672.0
4,539.0
5,573.0
6,1517.0
7,3354.0
8,4019.0
9,4534.0


**## RESULTADO Q2: MÉDIA DE PASSAGEIROS POR HORA QUE PEGARAM TAXI NO MES DE MAIO CONSIDERANDO AMBAS AS FROTAS.**

In [0]:
%sql
--Analise da media de passageiros por hora que pegaram taxi no mes de maio de ambas as frotas
with yellow as (
select
hour(tpep_pickup_datetime) AS hora,
round(sum(passenger_count_imputed),0) as qtd_passageiros
from df_y_med_pas_count
where  tpep_pickup_datetime >= date '2023-05-01'
and tpep_pickup_datetime <  date '2023-06-01'
group by 1
order by 1
),
green as (
select
hour(lpep_pickup_datetime) AS hora,
round(sum(passenger_count_imputed),0) as qtd_passageiros
from df_g_med_pas_count
where  lpep_pickup_datetime >= date '2023-05-01'
and lpep_pickup_datetime <  date '2023-06-01'
group by 1
order by 1
),
total as (
select
*
from yellow
union all
select
*
from green
)
select 
hora,
round(avg(qtd_passageiros),0) as media_passageiros
from total
group by 1
order by 1

hora,media_passageiros
0,67089.0
1,44027.0
2,28863.0
3,19011.0
4,12760.0
5,13200.0
6,31476.0
7,64310.0
8,88086.0
9,98819.0
